## Projeto PBD

In [1]:
import os
import sys

import numpy as np 
import pandas as pd  
import matplotlib.pyplot as plt
import seaborn as sns
# from ydata_profiling import ProfileReport
import warnings
warnings.filterwarnings("ignore")

In [2]:
import sys

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import *

from pyspark.ml import Pipeline
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder
from pyspark.ml.classification import LinearSVC
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [3]:
from pyspark.sql import SparkSession

# build our own SparkSession
spark = SparkSession\
    .builder\
    .appName("BigData")\
    .config("spark.sql.shuffle.partitions",6)\
    .config("spark.sql.repl.eagereval.enabled",True)\
    .getOrCreate() 

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/09 15:14:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark

### 1ªInicio
* Read CSV
* Perform EDA

	- Understand the problem
	- Which one makes sense to keep for Ranking

In [5]:
! head -n 2 train.csv
! tail -n 1 train.csv

date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,channel,srch_ci,srch_co,srch_adults_cnt,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,is_booking,cnt,hotel_continent,hotel_country,hotel_market,hotel_cluster
2014-08-11 07:46:59,2,3,66,348,48862,2234.2641,12,0,1,9,2014-08-27,2014-08-31,2,0,1,8250,1,0,3,2,50,628,1
2014-09-18 08:52:42,2,3,66,462,12565,106.4274,1198182,0,0,0,2014-09-18,2014-09-19,1,0,1,18811,1,1,1,2,50,592,42


In [6]:
filename = 'train.csv'
df_hotels =  spark.read.csv(filename, header=True, inferSchema="true", sep=',')

### Check DATA

In [7]:
df_hotels.printSchema()
df_hotels.show(1, vertical=True)
hotels_count = df_hotels.count()
hotels_count

root
 |-- date_time: timestamp (nullable = true)
 |-- site_name: integer (nullable = true)
 |-- posa_continent: integer (nullable = true)
 |-- user_location_country: integer (nullable = true)
 |-- user_location_region: integer (nullable = true)
 |-- user_location_city: integer (nullable = true)
 |-- orig_destination_distance: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- is_mobile: integer (nullable = true)
 |-- is_package: integer (nullable = true)
 |-- channel: integer (nullable = true)
 |-- srch_ci: timestamp (nullable = true)
 |-- srch_co: timestamp (nullable = true)
 |-- srch_adults_cnt: integer (nullable = true)
 |-- srch_children_cnt: integer (nullable = true)
 |-- srch_rm_cnt: integer (nullable = true)
 |-- srch_destination_id: integer (nullable = true)
 |-- srch_destination_type_id: integer (nullable = true)
 |-- is_booking: integer (nullable = true)
 |-- cnt: integer (nullable = true)
 |-- hotel_continent: integer (nullable = true)
 |-- hotel_country: 

37670293

### **Evaluate DATA**


## Criar nº de noites e ID_hotel

In [8]:
df_hotels = df_hotels.withColumn('Id_hotel', 
                    F.concat(F.col('hotel_cluster'), F.col('hotel_market'), F.col('hotel_country'), F.col('hotel_continent')))
df_hotels.show(1,vertical = True)


-RECORD 0----------------------------------------
 date_time                 | 2014-08-11 07:46:59 
 site_name                 | 2                   
 posa_continent            | 3                   
 user_location_country     | 66                  
 user_location_region      | 348                 
 user_location_city        | 48862               
 orig_destination_distance | 2234.2641           
 user_id                   | 12                  
 is_mobile                 | 0                   
 is_package                | 1                   
 channel                   | 9                   
 srch_ci                   | 2014-08-27 00:00:00 
 srch_co                   | 2014-08-31 00:00:00 
 srch_adults_cnt           | 2                   
 srch_children_cnt         | 0                   
 srch_rm_cnt               | 1                   
 srch_destination_id       | 8250                
 srch_destination_type_id  | 1                   
 is_booking                | 0                   


## Create new column number of nights

In [9]:
from pyspark.sql.functions import to_timestamp

df_hotels = df_hotels.withColumn("srch_co", to_timestamp(df_hotels.srch_co, "yyyy-MM-dd HH:mm:ss"))
df_hotels = df_hotels.withColumn("srch_ci", to_timestamp(df_hotels.srch_ci, "yyyy-MM-dd HH:mm:ss"))

from pyspark.sql.functions import datediff
from pyspark.sql.functions import unix_timestamp

#df_hotels = df_hotels.withColumn("num_nights", datediff(df_hotels.srch_co, df_hotels.srch_ci) - 1)
df_hotels = df_hotels.withColumn("num_nights", (unix_timestamp(df_hotels.srch_co) - unix_timestamp(df_hotels.srch_ci)) / 86400)
df_hotels = df_hotels.withColumn("num_nights", df_hotels.num_nights.cast("integer"))

In [10]:
df_hotels.show(1,vertical=True)

23/04/09 15:14:37 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
-RECORD 0----------------------------------------
 date_time                 | 2014-08-11 07:46:59 
 site_name                 | 2                   
 posa_continent            | 3                   
 user_location_country     | 66                  
 user_location_region      | 348                 
 user_location_city        | 48862               
 orig_destination_distance | 2234.2641           
 user_id                   | 12                  
 is_mobile                 | 0                   
 is_package                | 1                   
 channel                   | 9                   
 srch_ci                   | 2014-08-27 00:00:00 
 srch_co                   | 2014-08-31 00:00:00 
 srch_adults_cnt           | 2                   
 srch_children_cnt         | 0                   
 srch_rm_cnt        

## Casting number of nights and ID as integers

In [11]:
df_hotels = df_hotels.withColumn("num_nights", df_hotels.num_nights.cast("integer"))
df_hotels = df_hotels.withColumn("Id_hotel", df_hotels.Id_hotel.cast("integer"))

### Alteração do nome das variáveis

In [12]:
df_hotels = df_hotels.withColumnRenamed ('is_mobile', 'mobile').withColumnRenamed ('is_package','package').withColumnRenamed('srch_adults_cnt','num_adults').withColumnRenamed('srch_children_cnt','num_children').withColumnRenamed('srch_rm_cnt','num_room').withColumnRenamed('srch_destination_id','id_destination').withColumnRenamed('srch_destination_type_id','type_destination').withColumnRenamed('cnt','similar_srch').withColumnRenamed('is_booking','booking')

### **Cheking Number of NULLS**

In [13]:
print('\nNulls in Hotels:')
cols_to_forget = ['date_time','srch_ci','srch_co']
hotels_cols_interest = [x for x in df_hotels.columns if x not in cols_to_forget]
for cl in hotels_cols_interest:
    k = df_hotels.select(cl).filter(F.col(cl).isNull() | F.isnan(cl)).count()
    if k > 0:
        print(f'Column {cl} with {k} nulls or NaN, out of {hotels_count} records ({k*100/hotels_count:.2f}%)')


Nulls in Hotels:


Column orig_destination_distance with 13525001 nulls or NaN, out of 37670293 records (35.90%)


Column Id_hotel with 2723297 nulls or NaN, out of 37670293 records (7.23%)


Column num_nights with 47088 nulls or NaN, out of 37670293 records (0.13%)


## Checking How many orig_destination_distance are nulls per each hotel cluster

In [14]:
from pyspark.sql.functions import isnan, when, count, col

null_counts = df_hotels.groupBy('hotel_cluster').agg(count(when(isnan(col('orig_destination_distance')) | col('orig_destination_distance').isNull(), True)).alias('null_count'))
null_counts.show(10,vertical=True)

-RECORD 0---------------
 hotel_cluster | 10     
 null_count    | 139387 
-RECORD 1---------------
 hotel_cluster | 18     
 null_count    | 130773 
-RECORD 2---------------
 hotel_cluster | 25     
 null_count    | 215112 
-RECORD 3---------------
 hotel_cluster | 2      
 null_count    | 200439 
-RECORD 4---------------
 hotel_cluster | 46     
 null_count    | 332057 
-RECORD 5---------------
 hotel_cluster | 5      
 null_count    | 275276 
-RECORD 6---------------
 hotel_cluster | 59     
 null_count    | 232757 
-RECORD 7---------------
 hotel_cluster | 73     
 null_count    | 81326  
-RECORD 8---------------
 hotel_cluster | 38     
 null_count    | 146238 
-RECORD 9---------------
 hotel_cluster | 90     
 null_count    | 136184 
only showing top 10 rows



## Numero de vezes que os três primeiros hoteis cluster aparecem

In [15]:
count_10 = df_hotels.filter(df_hotels.hotel_cluster == 10).count()
print(count_10)
count_18 = df_hotels.filter(df_hotels.hotel_cluster == 18).count()
print(count_18)
count_25 = df_hotels.filter(df_hotels.hotel_cluster == 25).count()
print(count_25)


404764


545284


530591


## Removing orign_distance

In [16]:
hotels_cols_interest.remove('orig_destination_distance')
df_hotels = df_hotels.drop('orig_destination_distance')

In [17]:
df_hotels.show(1, vertical=True)

-RECORD 0------------------------------------
 date_time             | 2014-08-11 07:46:59 
 site_name             | 2                   
 posa_continent        | 3                   
 user_location_country | 66                  
 user_location_region  | 348                 
 user_location_city    | 48862               
 user_id               | 12                  
 mobile                | 0                   
 package               | 1                   
 channel               | 9                   
 srch_ci               | 2014-08-27 00:00:00 
 srch_co               | 2014-08-31 00:00:00 
 num_adults            | 2                   
 num_children          | 0                   
 num_room              | 1                   
 id_destination        | 8250                
 type_destination      | 1                   
 booking               | 0                   
 similar_srch          | 3                   
 hotel_continent       | 2                   
 hotel_country         | 50       

## Removing missing lines for Id_hotel and num_nights

In [18]:
df_hotels = df_hotels.na.drop()

In [19]:
df_hotels.show(1, vertical=True)

-RECORD 0------------------------------------
 date_time             | 2014-08-11 07:46:59 
 site_name             | 2                   
 posa_continent        | 3                   
 user_location_country | 66                  
 user_location_region  | 348                 
 user_location_city    | 48862               
 user_id               | 12                  
 mobile                | 0                   
 package               | 1                   
 channel               | 9                   
 srch_ci               | 2014-08-27 00:00:00 
 srch_co               | 2014-08-31 00:00:00 
 num_adults            | 2                   
 num_children          | 0                   
 num_room              | 1                   
 id_destination        | 8250                
 type_destination      | 1                   
 booking               | 0                   
 similar_srch          | 3                   
 hotel_continent       | 2                   
 hotel_country         | 50       

## Vendo quantos casos o numero de noites é menor que zero

In [20]:
df_filtered = df_hotels.filter(df_hotels.num_nights < 0).limit(3)
df_filtered.show(vertical=True)

-RECORD 0------------------------------------
 date_time             | 2014-12-03 08:08:44 
 site_name             | 11                  
 posa_continent        | 3                   
 user_location_country | 205                 
 user_location_region  | 354                 
 user_location_city    | 41495               
 user_id               | 7899                
 mobile                | 0                   
 package               | 0                   
 channel               | 9                   
 srch_ci               | 2015-11-10 00:00:00 
 srch_co               | 2015-09-15 00:00:00 
 num_adults            | 2                   
 num_children          | 0                   
 num_room              | 1                   
 id_destination        | 15254               
 type_destination      | 5                   
 booking               | 0                   
 similar_srch          | 1                   
 hotel_continent       | 5                   
 hotel_country         | 100      

In [21]:
total_rows = df_hotels.count()
df_filtered = df_hotels.filter(df_hotels.num_nights < 0)
filtered_rows = df_filtered.count()
percentage = (filtered_rows / total_rows) * 100
print("A porcentagem de casos em que num_nights é menor que 0 é de: {:.2f}%".format(percentage))

A porcentagem de casos em que num_nights é menor que 0 é de: 0.00%


In [22]:
df_filtered.count()

720

## Eliminando linhas cujo numero de noites é menor que 0

In [23]:
from pyspark.sql.functions import col

df_hotels = df_hotels.filter(~col("num_nights").isin([-1, 0]))

### **Summary to figure out outliers**

In [24]:
df_hotels.describe(hotels_cols_interest).show(vertical = True)

-RECORD 0-------------------------------------
 summary               | count                
 site_name             | 34759708             
 posa_continent        | 34759708             
 user_location_country | 34759708             
 user_location_region  | 34759708             
 user_location_city    | 34759708             
 user_id               | 34759708             
 mobile                | 34759708             
 package               | 34759708             
 channel               | 34759708             
 num_adults            | 34759708             
 num_children          | 34759708             
 num_room              | 34759708             
 id_destination        | 34759708             
 type_destination      | 34759708             
 booking               | 34759708             
 similar_srch          | 34759708             
 hotel_continent       | 34759708             
 hotel_country         | 34759708             
 hotel_market          | 34759708             
 hotel_cluste

In [25]:
df_hotels.select('hotel_cluster').distinct().count()

100

In [26]:
df_hotels.select('Id_hotel').distinct().count()

46225

### **UNIQUENESS**

In [27]:
print('\nUniqueness in Hotels:')
for cl in hotels_cols_interest:
    k = df_hotels.select(cl).distinct().count()
    if k > 0:
        print(f'Column {cl} with {k} distinct values, out of {hotels_count} records ({k*100/hotels_count:.2f}%)')


Uniqueness in Hotels:


Column site_name with 45 distinct values, out of 37670293 records (0.00%)


Column posa_continent with 5 distinct values, out of 37670293 records (0.00%)


Column user_location_country with 237 distinct values, out of 37670293 records (0.00%)


Column user_location_region with 1006 distinct values, out of 37670293 records (0.00%)


...

Column hotel_country with 211 distinct values, out of 37670293 records (0.00%)


Column hotel_market with 2110 distinct values, out of 37670293 records (0.01%)


Column hotel_cluster with 100 distinct values, out of 37670293 records (0.00%)


Column Id_hotel with 46225 distinct values, out of 37670293 records (0.12%)


Column num_nights with 465 distinct values, out of 37670293 records (0.00%)


# Saving clean data

## Saving a small dataset to analysis and train the model locally

In [28]:
seed = 5
with_replacement = False
fraction = 0.01          
small_df_hotels = df_hotels.sample(withReplacement=with_replacement, 
                                               fraction=fraction, seed=seed)
small_df_hotels.count()

347693

In [29]:
small_df_hotels.write.mode("overwrite").parquet("small-hotels")

23/04/09 15:23:01 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
23/04/09 15:23:01 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
23/04/09 15:23:01 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers


23/04/09 15:23:05 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
23/04/09 15:23:05 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
23/04/09 15:23:05 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
23/04/09 15:23:05 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
23/04/09 15:23:05 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers


23/04/09 15:23:09 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
23/04/09 15:23:09 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
23/04/09 15:23:09 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
23/04/09 15:23:09 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
23/04/09 15:23:09 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers


23/04/09 15:23:13 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
23/04/09 15:23:13 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


## Saving the whole dataset for training the model in AWS

In [30]:
hotels_cols_interest.remove('hotel_cluster')
hotels_cols_interest.remove('hotel_market')
hotels_cols_interest.remove('hotel_country')
hotels_cols_interest.remove('hotel_continent')
df_hotels = df_hotels.select(hotels_cols_interest) 

In [31]:
seed = 5
with_replacement = False
fraction = 1.0          
whole_df_hotels = df_hotels.sample(withReplacement=with_replacement, 
                                               fraction=fraction, seed=seed)
whole_df_hotels.count()

34759708

In [32]:
whole_df_hotels.write.mode("overwrite").parquet("whole-hotels")

23/04/09 15:23:26 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
23/04/09 15:23:26 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
23/04/09 15:23:26 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers


23/04/09 15:23:32 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
23/04/09 15:23:32 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
23/04/09 15:23:32 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
23/04/09 15:23:32 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
23/04/09 15:23:32 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
23/04/09 15:23:32 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


23/04/09 15:23:32 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
23/04/09 15:23:32 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
23/04/09 15:23:32 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
23/04/09 15:23:32 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
23/04/09 15:23:32 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
23/04/09 15:23:32 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
23/04/09 15:23:32 WARN MemoryManager: Total allocation exceeds 95.00

23/04/09 15:23:33 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
23/04/09 15:23:33 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
23/04/09 15:23:33 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
23/04/09 15:23:33 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


23/04/09 15:23:33 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
23/04/09 15:23:33 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
23/04/09 15:23:38 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
23/04/09 15:23:38 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
23/04/09 15:23:38 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
23/04/09 15:23:38 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers


23/04/09 15:23:39 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
23/04/09 15:23:39 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers


23/04/09 15:23:39 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
23/04/09 15:23:39 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
23/04/09 15:23:39 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
23/04/09 15:23:39 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
23/04/09 15:23:39 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers


23/04/09 15:23:39 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
23/04/09 15:23:39 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
23/04/09 15:23:39 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
23/04/09 15:23:39 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers


23/04/09 15:23:45 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
23/04/09 15:23:45 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers


23/04/09 15:23:45 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
23/04/09 15:23:45 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
